# Examples of highlighting a tensor

First, run the predude

In [ ]:
%run ../prelude.py --style=tree+uncompressed --animation=spacetime

## Get a tensor and display it

In [ ]:
t = Tensor("../../data/draw-b.yaml")
print(f"{t.getRoot():n*}")
print(t.getShape())

displayTensor(t)

## Highlight a single point in the tensor

Points in the tensor are indicated with a tuple (of length three since this is a rank-3 tensor)

In [ ]:
displayTensor(t, highlights=(2,1,3))

## Highlight two points 

Now the points are included in a list. This is actually the preferred form since sometimes there is ambiguity between a single point and a list of points.

In [ ]:
displayTensor(t, highlights=[(2,1,3), (4,1,3)])

## Highlight a subtree (1)

If a point doesn't have enough elements to get to a leaf payload, then a subtree is highlighted

In [ ]:
displayTensor(t, highlights=[(2,1,3),(4,1)])

## Highlight a subtree (2)

The fewer scalars in the tuple, the more of the tree that is highlighted. Note the comma needed to have a tuple with one element.

In [ ]:
displayTensor(t, highlights=[(2,1,3),(4,)])

## Associate a highlighted point with a PE

Use a dictionary with a PE name as a key to associate a point to highlight with a PE. In this case, the value of the key is a point.

In [ ]:
displayTensor(t, highlights={"PE0":(2,1,3)})

## Highlight points for multiple PEs

Use a dictionary with a muliple PE names as keys to associate points or lists of points with differnt PEs. Highlighted values for each PE is colored differently. Note, in this case, the value of one of the keys is a list of points and one is a subtensor not a single point.

Note again that using a list of points (as tuples) is the preferred form out a single point (tuple).

In [ ]:
displayTensor(t, highlights={"PE0":[(2,1,3), (4,)], "PE1":(2,3,3)})

## Highlight even more PEs

There are different colors for up to 8 PEs

In [ ]:
displayTensor(t, highlights={"PE0":(2,0,2), "PE1":(2,1,3),"PE2":(4,1,3), "PE3":(4,3,3)})

## Highlight the same point in two PEs

Note that PE0 and PE1 want to highlight the same point. So they share the value box.

In [ ]:
displayTensor(t, highlights={"PE0":(2,0,2), "PE1":(2,0,2),"PE2":(4,1,3), "PE3":(4,3,3)})

## Highlight the same point in even more PEs

Now all the PEs want to highlight the same point.

In [ ]:
displayTensor(t, highlights={"PE0":(2,0,2), "PE1":(2,0,2),"PE2":(2,0,2), "PE3":(2,0,2)})

## Highlight two overlapping subtensors in different PEs.

Now some subtensors share a value. Currently subtensor fiber highlighting does not distinguish which PE is highlighting the fiber. 

In [ ]:
displayTensor(t, highlights={"PE0":(2,0), "PE1":(2,)})

# The next group of examples use a wildcard as a coordinate.

The wildcard is indicated with a '?' as a coordinate and will match all coordinates at that rank. The final value will be highlighted if there are matching coordinates at all the other ranks.

## A rank-4 tensor

For wildcard experiments we sometimes use a rank-4 tensor

In [ ]:
t4 = Tensor.fromRandom(["A", "B", "C", "D"], [4, 4, 4, 4],(0.5, 0.5, 0.4, 0.5), 9, seed=10)

displayTensor(t4)

## Highlight a PE with a wildcard

Match any cordinate at the top rank and specific coordinates at the next two ranks.

Note, the highlight point must be in a list of points.

In [ ]:
displayTensor(t, highlights={"PE0":[('?',3, 2)]})

## Highlight a PE with a wildcard

Have wildcards in multiple ranks. This example will match any point with a 2 in the K-rank.

In [ ]:
displayTensor(t, highlights={"PE0":[('?','?',2)]})

## Highlight a subtensor with a wildcard

Highlight all the subtensors with a 3 in the second rank.

In [ ]:
displayTensor(t, highlights={"PE0":[('?',3)]})

## Highlight a subtensor with a wildcard

Highlight a subtensor with a 3 in the A and C ranks and anything in the B rank. Note, we're using a rank-4 tensor...

In [ ]:
displayTensor(t4, highlights={"PE0":[(3, '?', 3)]})

## Highlight a subtensor with a wildcard

In [ ]:
# I was going to do something

displayTensor(t4, highlights={"PE0":[('?', 3, 0)]})

## Highlight multiple points in a PE some with a wildcard

In [ ]:
displayTensor(t, highlights={"PE0":[('?','?',2), (4, 3, 3)]})

## Highlight a wildcard and a subtensor in a PE

Point (4,3,2) is included in the wildcard and the subtensor

In [ ]:
displayTensor(t, highlights={"PE0":[('?','?',2), (4, 3)]})

## Highlight points in multiple PEs - some with wildcard

Note same point is highlighted in one PE via a wildcard and another without

In [ ]:
displayTensor(t, highlights={"PE0":[('?','?',2)], "PE1":[(2, 0, 2)],"PE2":[(2,1,0)]})

## Highlight points in multiple PEs - with different wildcards

Note same point is highlighted in one PE via a wildcard, another with a different wildcard and then with an exact point.

In [ ]:
displayTensor(t, highlights={"PE0":[(2,'?',2)], "PE1":[('?', 0, 2)],"PE2":[(2,0,2)]})

## Highlight multiple points in each of multiple PEs with a wildcard

Note, the highlight point must be in a list of points.

In [ ]:
displayTensor(t, highlights={"PE0":[('?','?',2), (4, 3)], 
                             "PE1":[(2,'?',3)],
                             "PE2":[(4, 1, 3)],
                             "P33":[(2, 3, 2)]})

## Animate a system with mutiple PEs

Use **addActivity()** to animate a system with multiple PEs. For each parallel unit use **addActivity()** and specify the "worker" (i.e., PE name) that is causing the highlighted activity. The arguments after the "canvas" are points or lists of points to highlight for each tracked tensor for each PE (or "worker") corresponding to an iteration of the parallel loop. When the parallel activity has completed use **addFrame()** to aggreate all the highlights for the separate PEs.

In [ ]:
# Use a value in A to scale all elements in B and sum

a = Tensor.fromUncompressed(["X"], [1, 2, 3, 4, 5, 6, 7, 8])
a.setName("A").setColor("Purple")

b = Tensor.fromUncompressed(["Y"], [1, 2, 3, 4])
b.setName("B").setColor("Green")

z = Tensor(rank_ids=["X"], name="Z")

a_x = a.getRoot()
b_y = b.getRoot()
z_x = z.getRoot()

# Specifiy the tensor to track (and display) on the canvas
canvas=createCanvas(a, b, z)

for x, (z_ref, a_val) in z_x << a_x:
    # Assume all elements in b_y are processed in parallel
    for y, b_val in b_y:
        pe = y
        z_ref += a_val *b_val
        # Add activity to display for each tracked tensor and the current worker
        addActivity(canvas, (x,), (y,), (x,), worker=f"PE{pe}")
    addFrame(canvas)

displayCanvas(canvas)

## Program illustrating parallel activity in 2-D tensors

Use a rank of the tensor to correspond to the parallel activity

In [ ]:
a = Tensor.fromUncompressed(["X", "Y"], [[1, 2], [3, 4]])
a.setName("A").setColor("Purple")

b = Tensor.fromUncompressed(["X", "Y"], [[5, 6], [7, 8]])
b.setName("B").setColor("Green")

z = Tensor(rank_ids=["X","Y"], name="Z")

displayTensor(a)
displayTensor(b)
displayTensor(z)

a_x = a.getRoot()
b_x = b.getRoot()
z_x = z.getRoot()

canvas=createCanvas(a, b, z)

for x, (z_y, (a_y, b_y)) in z_x << (a_x & b_x):
    for y, (z_ref, (a_val, b_val)) in z_y << (a_y & b_y):
        pe = f"PE{y%2}"
        z_ref += a_val + b_val
        addActivity(canvas, (x,y), (x,y), (x,y), worker=pe)
    addFrame(canvas)

displayCanvas(canvas)

## Program illustrating parallel activity

Parallelism is created by explicitly tiling the tensor

In [ ]:
# Program to sum to tensors

a = Tensor.fromUncompressed(["X"], [1, 2, 3, 4])
a.setName("A").setColor("Purple")

b = Tensor.fromUncompressed(["X"], [5, 6, 7, 8])
b.setName("B").setColor("Green")

a_split = a.splitUniform(2, relativeCoords=True)
b_split = b.splitUniform(2, relativeCoords=True)
                                    
z = Tensor(rank_ids=["X1","X0"], name="Z")

displayTensor(a_split)
displayTensor(b_split)
displayTensor(z)

a_x1 = a_split.getRoot()
b_x1 = b_split.getRoot()
z_x1 = z.getRoot()

canvas=createCanvas(a_split, b_split, z)

for x1, (z_x0, (a_x0, b_x0)) in z_x1 << (a_x1 & b_x1):
    for x0, (z_ref, (a_val, b_val)) in z_x0 << (a_x0 & b_x0):
        pe = f"PE{x0}"
        z_ref += a_val + b_val
        addActivity(canvas, (x1,x0), (x1,x0), (x1,x0), worker=pe)
    addFrame(canvas)

displayCanvas(canvas)

## Program illustrating parallel activity (with sparsity)

Parallelism is created by explicitly tiling the tensor

In [ ]:
# Program to sum to tensors

a = Tensor.fromUncompressed(["X"], [1, 0, 3, 4, 5, 6, 7, 8])
a.setName("A").setColor("Purple")

b = Tensor.fromUncompressed(["X"], [11,12 ,0, 14, 15, 16, 17, 0])
b.setName("B").setColor("Green")

a_split = a.splitUniform(2, relativeCoords=True)
b_split = b.splitUniform(2, relativeCoords=True)
                                    
z = Tensor(rank_ids=["X1","X0"], name="Z")

displayTensor(a_split)
displayTensor(b_split)
displayTensor(z)

a_x1 = a_split.getRoot()
b_x1 = b_split.getRoot()
z_x1 = z.getRoot()

canvas=createCanvas(a_split, b_split, z)

for x1, (z_x0, (a_x0, b_x0)) in z_x1 << (a_x1 & b_x1):
    for x0, (z_ref, (a_val, b_val)) in z_x0 << (a_x0 & b_x0):
        pe = f"PE{x0}"
        z_ref += a_val + b_val
        addActivity(canvas, (x1,x0), (x1,x0), (x1,x0), worker=pe)
    addFrame(canvas)

displayCanvas(canvas)